In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')
! cp /content/drive/Sharedwithme/CSCE5214Project1/github_keys/keys* /root/.ssh/

cp: cannot stat '/content/drive/Sharedwithme/CSCE5214Project1/github_keys/keys*': No such file or directory


In [ ]:
import os

# Checking if each directory or file exists
if os.path.exists('/content/drive'):
    print("'/content/drive' exists")
else:
    print("'/content/drive' does not exist")

if os.path.exists('/content/drive/Shared with me'):
    print("'/content/drive/Shared with me' exists")
else:
    print("'/content/drive/Shared with me' does not exist")

if os.path.exists('/content/drive/Shared with me/CSCE5214Project1'):
    print("'/content/drive/Shared with me/CSCE5214Project1' exists")
else:
    print("'/content/drive/Shared with me/CSCE5214Project1' does not exist")

if os.path.exists('/content/drive/Shared with me/CSCE5214Project1/github_keys'):
    print("'/content/drive/Shared with me/CSCE5214Project1/github_keys' exists")
else:
    print("'/content/drive/Shared with me/CSCE5214Project1/github_keys' does not exist")

if os.path.exists('/content/drive/Shared with me/CSCE5214Project1/github_keys/keys*'):
    print("Keys file exists")
else:
    print("Keys file does not exist")

'/content/drive' does not exist
'/content/drive/Shared with me' does not exist
'/content/drive/Shared with me/CSCE5214Project1' does not exist
'/content/drive/Shared with me/CSCE5214Project1/github_keys' does not exist
Keys file does not exist


In [ ]:
import pandas as pd
import numpy as np

## Load Dataset and Clean Dataset

In [ ]:
# read the dataset
df = pd.read_csv("train.csv", sep = ",")
df.info()

FileNotFoundError: [Errno 2] No such file or directory: 'train.csv'

In [ ]:
# drop the last 50,000 rows
df = df.iloc[:-50000]

# rename column name
df.rename(columns = {'opened_position_qty ': 'opened_position_qty'}, inplace = True)

In [ ]:
# check for missing data
df.isnull().sum()

In [ ]:
# replace data points based on surrounding data values
# linear assumes a 'straight line' from the previous data points to the next one. may be less accurate if we have a bunch of missing data points in a row
df['opened_position_qty'].interpolate(method='linear', inplace=True)
df['closed_position_qty'].interpolate(method='linear', inplace=True)

In [ ]:
# check for missing data
df.isnull().sum()

In [ ]:
df.head()

In [ ]:
# remove the first row which is the only one with missing data now
df = df.iloc[1:]

In [ ]:
df.describe()

In [ ]:
df.head()

In [ ]:
df.tail()

## Split the Dataset for Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split

# predictors
X = df.drop(columns = ['y'])

# response variable
y = df['y']

# spilt the dataset into training and testing with a 80/20 spilt with a random state
X_train, X_test, y_train, y_test, = train_test_split(X, y, test_size = 0.2, random_state = 123)

Long Short-Term Memory Model

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [ ]:
# min-max scaler - Xscaled = (X - Xmin) / (Xmax - Xmin) - may not be optimal for our data idrk
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# using numPy to reshape data. LSTM requires 3D data - samples (individual data points(rows)), time steps (length of time step used to predict target var), features (predictor columns used for making predictions)
# default of 1 time step, might set this to 10 or something.
X_scaled = np.reshape(X_scaled, (X_scaled.shape[0], 1, X_scaled.shape[1]))
X_train_LSTM, X_test_LSTM, y_train_LSTM, y_test_LSTM, = train_test_split(X_scaled, y, test_size = 0.2, random_state = 69)

In [ ]:
# Build LSTM
LSTM_model = Sequential()

# adding first layer. Dropout sets a # of neurons to zero during training to prevent overreliance on certain patterns. (overfitting)
# "X_train.shape[1], X_train.shape[2]" selects the time steps and the # features as the 'input shape' so the model knows what data it's getting
LSTM_model.add(LSTM(units = 50, return_sequences = True, input_shape = (X_train.shape[1], X_train.shape[2])))
LSTM_model.add(Dropout(0.1)) # Not sure what a good value here would be

# second layer, adds 50 more neurons
LSTM_model.add(LSTM(units = 50, return_sequences = False))
LSTM_model.add(Dropout(0.1))

# output layer
LSTM_model.add(Dense(units = 1, activation = 'sigmoid'))

# compiles model w selected features. The optimizer is how the model updates weights using backpropagation calc and is (variant of SGD - stochastic gradient descent) - loss function tells optimizer how far guesses were off.
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=[AUC(name='auc')])

history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))

# Evaluate the model on test data
test_loss, test_auc = model.evaluate(X_test, y_test)
print(f"Test Loss: {test_loss}")
print(f"Test AUC: {test_auc}")



```
# This is formatted as code
```

## Random Forest Model

### Fiting a Random Forest

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

# parameters for hypertuning
parameters = [{'n_estimators': [100, 500, 1000, 2000],
               'max_features': ['sqrt'],
               'max_depth': [3, 5, 7, 10, 15, 20],
               'criterion': ['gini', 'entropy']}]

# hypertune for random froest
GridSearch_RandomForest = GridSearchCV(estimator = RandomForestClassifier(random_state = 123), param_grid = parameters, scoring = 'accuracy', cv = 5, n_jobs = -1)

# fit a random forest
GridSearch_RandomForest.fit(X_train, y_train)
print(GridSearch_RandomForest.best_params_)

# score for the best model
print(GridSearch_RandomForest.score(X_train, y_train))


### Predicted Data from Random Forest

In [ ]:
prediction_RandomForest = GridSearch_RandomForest.predict(X_test)
prediction_probs_RandomForest = GridSearch_RandomForest.predict_proba(X_test)